# 1. Functions and OOP

## 1.A Reviewing functions

In [1]:
#src/examples/function_basic.py
# Function 1
def string_print(a):
    return a

result = string_print('LOOK MOM I CAN PRINT')
print(f"The result of string_print function: {result}")

# Function 2
def add_func(a, b, c):
    final = a + b + c
    return final

result = add_func(5, 3, 2)
print(f"The result of add_func function: {result}")

The result of string_print function: LOOK MOM I CAN PRINT
The result of add_func function: 10


## 1.1 Functions: Downloading and transforming

### 1.1.1 Download

In [2]:
import pandas as pd
import requests
import io
import os

def download_macro_data(url):
    """Downloads macroeconomic data from a URL and returns a pandas DataFrame."""
    print(f"Fetching macroeconomic data from: {url}")
    df = pd.read_stata(url)
    return df

def download_oecd_data(url):
    """Downloads OECD data from a URL and returns a pandas DataFrame."""
    print(f"Fetching OECD data from: {url}")
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = io.StringIO(response.text)
    df = pd.read_csv(data)
    return df

url_macro = 'https://github.com/KMueller-Lab/Global-Macro-Database/raw/refs/heads/main/data/final/chainlinked_infl.dta'
url_oecd = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_PDB@DF_PDB_ULC_Q,1.0/.Q.......?startPeriod=1990-Q4&format=csv"

df_macro_raw = download_macro_data(url_macro)
df_oecd_raw = download_oecd_data(url_oecd)

Fetching macroeconomic data from: https://github.com/KMueller-Lab/Global-Macro-Database/raw/refs/heads/main/data/final/chainlinked_infl.dta
Fetching OECD data from: https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_PDB@DF_PDB_ULC_Q,1.0/.Q.......?startPeriod=1990-Q4&format=csv


### 1.1.2 Filter

In [3]:
def filter_rename_macro_nz(df):
    """Filters macroeconomic data for New Zealand, selects and renames columns."""
    df_nz = df.query("ISO3 == 'NZL'")[['ISO3', 'year', 'OECD_KEI_infl', 'BIS_infl']].dropna()
    df_nz = df_nz.rename({"ISO3":'country', "year":'date'}, axis=1)
    return df_nz

def filter_rename_oecd_nz(df):
    """Filters OECD data for New Zealand, selects, renames, and drops columns."""
    cols = ['REF_AREA', 'TIME_PERIOD', 'OBS_VALUE', 'MEASURE', 'UNIT_MEASURE']
    df_nz = df[cols].query("REF_AREA == 'NZL' & MEASURE=='ULCE' & UNIT_MEASURE == 'PA'")
    print(f"Filtered OECD data for NZL with shape: {df_nz.shape}")
    df_nz = df_nz.rename({"REF_AREA":'country', "TIME_PERIOD":'date', 'OBS_VALUE':'ULCE'}, axis=1).drop(["MEASURE", "UNIT_MEASURE"], axis=1)
    return df_nz

In [4]:
df_macro_nz = filter_rename_macro_nz(df_macro_raw.copy())
df_oecd_nz = filter_rename_oecd_nz(df_oecd_raw.copy())

Filtered OECD data for NZL with shape: (136, 5)


## 1.3 Convert datetime

In [5]:
def convert_datetime(df_oecd, df_macro):
    """Converts date columns to datetime objects."""
    df_oecd['date'] = pd.PeriodIndex(df_oecd['date'], freq='Q').to_timestamp().date
    print("Converted 'date' column in OECD data for NZL to datetime.date.")
    print(f"Data type of 'date' in OECD data for NZL: {df_oecd['date'].dtype}")
    df_macro['date'] = pd.to_datetime(df_macro['date'], format = '%Y').dt.date
    print("Converted 'date' column in macroeconomic data for NZL to datetime.date.")
    print(f"Data type of 'date' in macroeconomic data for NZL: {df_macro['date'].dtype}")
    return df_oecd, df_macro

In [6]:
df_oecd_nz_dt, df_macro_nz_dt = convert_datetime(df_oecd_nz.copy(), df_macro_nz.copy())

Converted 'date' column in OECD data for NZL to datetime.date.
Data type of 'date' in OECD data for NZL: object
Converted 'date' column in macroeconomic data for NZL to datetime.date.
Data type of 'date' in macroeconomic data for NZL: object


## 1.4 Set index

In [7]:
def set_index(df_oecd, df_macro):
    """Sets 'country' and 'date' as the index for both DataFrames."""
    df_macro = df_macro.set_index(['country', 'date'])
    df_oecd = df_oecd.set_index(['country', 'date'])
    return df_oecd, df_macro
    
df_oecd_indexed, df_macro_indexed = set_index(df_oecd_nz_dt.copy(), df_macro_nz_dt.copy())

## 1.5 Merge

In [8]:
def merge_data(df_macro, df_oecd):
    """Merges two DataFrames based on their index."""
    df_merge = pd.merge(
        df_macro,
        df_oecd,
        right_index = True,
        left_index = True,
        how = 'inner'
    )
    return df_merge
df_merged = merge_data(df_macro_indexed.copy(), df_oecd_indexed.copy())

## 1.6 Export

In [9]:
def export_data(df_merged, df_oecd_raw):
    """Exports the merged and raw OECD DataFrames to CSV files."""
    os.makedirs("data/intermediate/", exist_ok=True)
    df_merged.to_csv("data/intermediate/merged_data_nz.csv")
    print("Exported merged DataFrame to data/intermediate/merged_data_nz.csv")
    os.makedirs("data/raw/", exist_ok=True)
    df_oecd_raw.to_csv("data/raw/oecd.csv")
    print("Exported raw OECD DataFrame to data/raw/oecd.csv")
export_data(df_merged.copy(), df_oecd_raw.copy())

Exported merged DataFrame to data/intermediate/merged_data_nz.csv
Exported raw OECD DataFrame to data/raw/oecd.csv


# 2. OOP: Making python files which use multiple tools (functions)

## 2.A OOP Basic Example

In [10]:
class SimpleOperationPerformer:
    def perform_string_print(self, input_string):
        print("Running perform_string_print...")
        result = input_string
        print(f"The result of string_print function: {result}")
        return result

    def perform_addition(self, num1, num2, num3):
        print("Running perform_addition...")
        final = num1 + num2 + num3
        print(f"The result of add_func function: {final}")
        return final

    def run_all_operations(self):
        print("\nStarting run_all_operations...")
        self.perform_string_print('LOOK MOM I CAN PRINT')
        self.perform_addition(5, 3, 2)
        print("Finished run_all_operations.")

In [11]:
# Let's create our SimpleOperationPerformer tool
ot = SimpleOperationPerformer()

In [12]:
# Now, let's tell it to run each action individually
print("\nRunning operations individually:")
ot.perform_string_print('Hello from individual call!')
ot.perform_addition(10, 20, 30)

# And let's also run all operations together
ot.run_all_operations()


Running operations individually:
Running perform_string_print...
The result of string_print function: Hello from individual call!
Running perform_addition...
The result of add_func function: 60

Starting run_all_operations...
Running perform_string_print...
The result of string_print function: LOOK MOM I CAN PRINT
Running perform_addition...
The result of add_func function: 10
Finished run_all_operations.


In [13]:
# --- Relating to pandas ---
# In pandas, you often create a 'DataFrame' which is like a table.
# Our 'SimpleOperationPerformer' (now called 'ot') is like a tool
# that can do things with data, similar to how you can perform
# operations on a pandas DataFrame (like calculating sums or printing parts).

# For example, if a pandas DataFrame was called 'df', you might do:
# print(df.head())  # This is like our 'perform_string_print' showing the start.
# total = df['column_name'].sum() # This is like our 'perform_addition' calculating a total.

# 'ot' is a simple tool we made, while pandas provides more powerful tools
# for working with tables of data.

## 2.2 Create tool to download and transform our data like in 1.1

In [14]:
# Think of '__init__' as the special setup step when you create a new startup tool.
# It's like the instructions for getting the tool ready to use.

class DataProcessor:
    # '__init__' is a special function that runs automatically when you make a DataProcessor tool.
    def __init__(self, intermediate_folder="data/intermediate/", raw_folder="data/raw/"):
        # Inside '__init__', we're giving our new startup tool some initial settings.
        # 'self' here just refers to the specific DataProcessor tool you are creating.
        # We're setting up where this tool will look for and store its information.
        self.intermediate_folder = intermediate_folder  # This tells the tool where to put partially finished data.
        self.raw_folder = raw_folder                    # This tells the tool where the original data is kept.

# Let's say you want to create a DataProcessor startup tool to organize your files.
# When you do this: The '__init__' function automatically runs. It configures the 'intermediate_folder'
dp = DataProcessor()

# Now, if you want to know where this specific 'dp' tool is storing its intermediate files, you can ask:
dp.intermediate_folder

'data/intermediate/'

In [15]:
# Once we create this dp we can see the stored self information 
dp.raw_folder

'data/raw/'

## 2.3 Init plus another tool

In [16]:
# Think of '__init__' as the special setup step when you create a new startup tool.
# It's like the instructions for getting the tool ready to use.

import pandas as pd
import requests
import io

class DataProcessor:
    # '__init__' is a special function that runs automatically when you make a DataProcessor tool.
    def __init__(self, intermediate_folder="intermediate/", raw_folder="raw/"):
        # Inside '__init__', we're giving our new startup tool some initial settings.
        # 'self' here just refers to the specific DataProcessor tool you are creating.
        # We're setting up where this tool will look for and store its information.
        self.intermediate_folder = intermediate_folder  # This tells the tool where to put partially finished info.
        self.raw_folder = raw_folder                    # This tells the tool where the original info is kept.
        self.df_macro = None                             # This will hold macro info after we download it.
        self.df_oecd = None                              # This will hold OECD info after we download it.

    # This is a new ability for our tool: downloading macro info from a website.
    def download_macro(self, macro_url):
        # We use a special tool (pd.read_stata) to read the info directly from the web address.
        self.df_macro = pd.read_stata(macro_url)
        # Once downloaded, we store the info within our tool and then give it back.
        return self.df_macro

    # Here's another new ability: downloading info from the OECD website.
    def download_oecd(self, oecd_url):
        # We ask the website for the info.
        response = requests.get(oecd_url)
        # If something went wrong while asking, this will let us know.
        response.raise_for_status()
        # We take the text info we got and treat it like a file.
        data = io.StringIO(response.text)
        # Then, we use another special tool (pd.read_csv) to understand this info.
        self.df_oecd = pd.read_csv(data)
        # We store this info in our tool and then give it back.
        return self.df_oecd

# Let's say you want to create a DataProcessor startup tool to organize your files and download data.
# When you do this: The '__init__' function automatically runs, setting up the initial storage locations
dp = DataProcessor()

# Now, we're telling our 'dp' tool to download data from the provided web addresses:
macro_df = dp.download_macro(
    macro_url = 'https://github.com/KMueller-Lab/Global-Macro-Database/raw/refs/heads/main/data/final/chainlinked_infl.dta'
)
dp.df_macro.head(2)

,ISO3,year,ADB_infl,AHSTAT_infl,AMF_infl,BCEAO_infl,BIS_infl,BORDO_infl,CEPAC_infl,EUS_infl,...,OECD_KEI_infl,WB_CC_infl,WDI_infl,WDI_ARC_infl,CS1_infl,CS2_infl,infl,chainlinking_ratio,source,source_change
0,ZWE,2029.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.131044,1.0,IMF_WEO,NaN
1,ZWE,2028.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.108963,1.0,IMF_WEO,NaN


In [17]:
# The 'download_macro' ability is used to get the macro information, and we're saving
# the downloaded info into a new container called 'macro_df'.

oecd_df = dp.download_oecd(
    oecd_url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_PDB@DF_PDB_ULC_Q,1.0/.Q.......?startPeriod=1990-Q4&format=csv"
)
dp.df_oecd.head(2)


,DATAFLOW,REF_AREA,FREQ,MEASURE,ACTIVITY,UNIT_MEASURE,PRICE_BASE,TRANSFORMATION,ADJUSTMENT,CONVERSION_TYPE,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,BASE_PER,DECIMALS
0,OECD.SDD.TPS:DSD_PDB@DF_PDB_ULC_Q(1.0),DNK,Q,LCEMP,_T,PA,V,GY,S,NC,2017-Q3,1.793888,A,0,NaN,2
1,OECD.SDD.TPS:DSD_PDB@DF_PDB_ULC_Q(1.0),DNK,Q,LCEMP,_T,PA,V,GY,S,NC,2017-Q2,1.108080,A,0,NaN,2


## 2.4 Convert all of 1.2 to a python file

In [21]:
# Think of '__init__' as the special setup step when you create a new startup tool.
# It's like the instructions for getting the tool ready to use.

import pandas as pd
import requests
import io
import os

class DataProcessor:
    # '__init__' is a special function that runs automatically when you make a DataProcessor tool.
    def __init__(self, intermediate_folder="intermediate/", raw_folder="raw/"):
        # Inside '__init__', we're giving our new startup tool some initial settings.
        # 'self' here just refers to the specific DataProcessor tool you are creating.
        # We're setting up where this tool will look for and store its information.
        self.intermediate_folder = intermediate_folder  # Where we'll put partially finished info.
        self.raw_folder = raw_folder                    # Where the original info is kept.
        self.df_macro = None                             # Will hold macro info after download.
        self.df_oecd = None                              # Will hold OECD info after download.
        self.df_macro_nz = None                          # Will hold filtered and renamed NZ macro info.
        self.df_oecd_nz = None                           # Will hold filtered and renamed NZ OECD info.
        self.df_merged = None                            # Will hold the combined info.

    # This is a new ability for our tool: downloading macro info from a website.
    def download_macro(self, macro_url):
        print("Running download_macro...")
        self.df_macro = pd.read_stata(macro_url)
        print("Macro data downloaded successfully.")
        return self.df_macro

    # Here's another new ability: downloading info from the OECD website.
    def download_oecd(self, oecd_url):
        print("Running download_oecd...")
        response = requests.get(oecd_url)
        response.raise_for_status()
        data = io.StringIO(response.text)
        self.df_oecd = pd.read_csv(data)
        print("OECD data downloaded successfully.")
        return self.df_oecd

    # This ability filters the macro info to only include data for New Zealand (NZL),
    # selects specific columns, removes any missing data, and renames the columns.
    def filter_rename_macro_nz(self):
        print("Running filter_rename_macro_nz...")
        self.df_macro_nz = self.df_macro.query("ISO3 == 'NZL'")[['ISO3', 'year', 'OECD_KEI_infl', 'BIS_infl']].dropna().copy()
        self.df_macro_nz.rename({"ISO3":'country', "year":'date'}, axis=1, inplace=True)
        print("Macro data filtered and renamed for NZ.")
        return self.df_macro_nz

    # This ability filters the OECD info to only include data for New Zealand (NZL)
    # for a specific measure ('ULCE') and unit ('PA'), selects relevant columns,
    # renames them, and removes unnecessary columns.
    def filter_rename_oecd_nz(self):
        print("Running filter_rename_oecd_nz...")
        cols_oecd = ['REF_AREA', 'TIME_PERIOD', 'OBS_VALUE', 'MEASURE', 'UNIT_MEASURE']
        self.df_oecd_nz = self.df_oecd[cols_oecd].query("REF_AREA == 'NZL' & MEASURE=='ULCE' & UNIT_MEASURE == 'PA'").copy()
        self.df_oecd_nz.rename({"REF_AREA":'country', "TIME_PERIOD":'date', 'OBS_VALUE':'ULCE'}, axis=1, inplace=True)
        self.df_oecd_nz.drop(["MEASURE", "UNIT_MEASURE"], axis=1, inplace=True)
        print("OECD data filtered and renamed for NZ.")
        return self.df_oecd_nz

    # This ability converts the 'date' column in the New Zealand macro info
    # from a year format to a standard date format.
    def convert_datetime_macro_nz(self):
        print("Running convert_datetime_macro_nz...")
        self.df_macro_nz['date'] = pd.to_datetime(self.df_macro_nz['date'], format = '%Y').dt.date
        print("Macro dates converted to datetime.")
        return self.df_macro_nz

    # This ability converts the 'date' column in the New Zealand OECD info
    # from a quarterly format (like '1990-Q4') to a standard date format.
    def convert_datetime_oecd_nz(self):
        print("Running convert_datetime_oecd_nz...")
        self.df_oecd_nz['date'] = pd.PeriodIndex(self.df_oecd_nz['date'], freq='Q').to_timestamp().date
        print("OECD dates converted to datetime.")
        return self.df_oecd_nz

    # This ability sets the 'country' and 'date' columns as the main identifiers (index)
    # for the New Zealand macro info. This helps in combining data later.
    def set_index_macro_nz(self):
        print("Running set_index_macro_nz...")
        self.df_macro_nz.set_index(['country', 'date'], inplace=True)
        print("Macro data index set to country and date.")
        return self.df_macro_nz

    # This ability does the same as above, but for the New Zealand OECD info.
    def set_index_oecd_nz(self):
        print("Running set_index_oecd_nz...")
        self.df_oecd_nz.set_index(['country', 'date'], inplace=True)
        print("OECD data index set to country and date.")
        return self.df_oecd_nz

    # This ability combines the New Zealand macro and OECD info into a single table
    # based on matching 'country' and 'date'. It only keeps the data that exists in both tables.
    def merge_data(self):
        print("Running merge_data...")
        self.df_merged = pd.merge(
            self.df_macro_nz,
            self.df_oecd_nz,
            right_index = True,
            left_index = True,
            how = 'inner'
        )
        print("Macro and OECD data merged.")
        return self.df_merged

    # This ability handles exporting our processed data to files.
    def export_data(self, output_dir_intermediate="data/intermediate/processed/", output_dir_raw="data/raw/"):
        print("Running export_data...")
        os.makedirs(output_dir_intermediate, exist_ok=True)
        merged_filepath = os.path.join(output_dir_intermediate, "merged_data_nz.csv")
        self.df_merged.to_csv(merged_filepath)
        print(f"Merged data exported to: {merged_filepath}")

        os.makedirs(output_dir_raw, exist_ok=True)
        raw_oecd_filepath = os.path.join(output_dir_raw, "oecd_raw.csv")
        self.df_oecd.to_csv(raw_oecd_filepath)
        print(f"Raw OECD data exported to: {raw_oecd_filepath}")


In [19]:
# Let's create our DataProcessor tool.
processor = DataProcessor()

# Now, let's define the web addresses for our data:
macro_data_url = 'https://github.com/KMueller-Lab/Global-Macro-Database/raw/refs/heads/main/data/final/chainlinked_infl.dta'
oecd_data_url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_PDB@DF_PDB_ULC_Q,1.0/.Q.......?startPeriod=1990-Q4&format=csv"

# Now, we will run each method of our 'processor' tool one by one.

# 1. Download macro data
macro_df = processor.download_macro(macro_data_url)

# 2. Download OECD data
oecd_df = processor.download_oecd(oecd_data_url)

# 3. Filter and rename macro data for New Zealand
macro_nz_df = processor.filter_rename_macro_nz()

# 4. Filter and rename OECD data for New Zealand
oecd_nz_df = processor.filter_rename_oecd_nz()

# 5. Convert datetime in macro data
macro_nz_df_dt = processor.convert_datetime_macro_nz()

# 6. Convert datetime in OECD data
oecd_nz_df_dt = processor.convert_datetime_oecd_nz()

# 7. Set index for macro data
macro_nz_df_indexed = processor.set_index_macro_nz()

# 8. Set index for OECD data
oecd_nz_df_indexed = processor.set_index_oecd_nz()

# 9. Merge the two datasets
merged_df = processor.merge_data()

# 10. Export the processed and raw data
processor.export_data()

Running download_macro...
Macro data downloaded successfully.
Running download_oecd...
OECD data downloaded successfully.
Running filter_rename_macro_nz...
Macro data filtered and renamed for NZ.
Running filter_rename_oecd_nz...
OECD data filtered and renamed for NZ.
Running convert_datetime_macro_nz...
Macro dates converted to datetime.
Running convert_datetime_oecd_nz...
OECD dates converted to datetime.
Running set_index_macro_nz...
Macro data index set to country and date.
Running set_index_oecd_nz...
OECD data index set to country and date.
Running merge_data...
Macro and OECD data merged.
Running export_data...
Merged data exported to: intermediate/processed/merged_data_nz.csv
Raw OECD data exported to: raw/oecd_raw.csv


# 3.A Creating python files outside of the notebooks

### 3.A.1 This notebook and the files are "far" away

In [32]:
import os
#Our python files are always in the src folder
os.listdir("../../../src/examples/")

['function_import_data.py',
 'oop_basic.py',
 'oop_manipulate_save.py',
 'oop_import_data_notebook.py',
 'oop_import_data.py',
 '__pycache__',
 'function_manipulate_save.py',
 'function_basic.py']

### 2.A.2 We must "add" the location of the python files to use

In [23]:
import sys
sys.path.append("../../../src/examples/")

## 3.1 Importing functions from python file

In [24]:
# Import functions individual
from function_basic import string_print
string_print('yes')

The result of string_print function: LOOK MOM I CAN PRINT
The result of add_func function: 10


'yes'

In [25]:
from function_basic import add_func
add_func(5, 3, 2)

10

In [26]:
# Or import all the functions from one specific python file
# Restart the kernal and run this it should work
from function_basic import *
add_func(5, 3, 2)
string_print('yes')

'yes'

## 3.2 Sometimes it is tricky to run python files from notebook

In [33]:
#from function_import_data import *

## 3.3. OOP: The type of files we like to run from notebooks

In [34]:
# Import function
from oop_basic import SimpleOperationPerformer

In [35]:
# Create tool: This is the same as the Pandas library
my_pd = SimpleOperationPerformer()

In [36]:
# Use the tools in the my_pd library
my_pd.perform_string_print('Hello from individual call!')
my_pd.perform_addition(10, 20, 30)

Running perform_string_print...
The result of string_print function: Hello from individual call!
Running perform_addition...
The result of add_func function: 60


60

## 3.4 OOP: More advanced

### 3.4.1 Again it's tricky to run from notebook

In [37]:
from oop_import_data import SimpleFileDataProcessor
# Let's create our SimpleFileDataProcessor tool
processor = SimpleFileDataProcessor(
    file_location="data/examples/module_1/week_1"
)

# Now, let's tell it to do its job!
processor.run_all_steps()

Current python folder : /Users/corybaird/Desktop/graspp_2025_spring/src/examples
Folder where all our files are located (base/root folder): /Users/corybaird/Desktop/graspp_2025_spring
Data folder: /Users/corybaird/Desktop/graspp_2025_spring/data
Starting the file processing...
This file is run from the following path:


/Users/corybaird/Desktop/graspp_2025_spring/notebooks/module_1/week_3


Looking inside this folder: data/examples/module_1/week_1


FileNotFoundError: [Errno 2] No such file or directory: 'data/examples/module_1/week_1'

### 3.4.2 Must create some hacks to make the python file run

In [38]:
import sys
sys.path.append("../../../src/examples/")
from oop_import_data_notebook import SimpleFileDataProcessor
# Let's create our SimpleFileDataProcessor tool
processor = SimpleFileDataProcessor(
    file_location="data/examples/module_1"
)

# Now, let's tell it to do its job!
processor.run_all_steps()

Current python folder : /Users/corybaird/Desktop/graspp_2025_spring/src/examples
Folder where all our files are located (base/root folder): /Users/corybaird/Desktop/graspp_2025_spring
Data folder: /Users/corybaird/Desktop/graspp_2025_spring/data
Starting the file processing...
This file is run from the following path:


/Users/corybaird/Desktop/graspp_2025_spring/notebooks/module_1/week_3


Looking inside this folder: /Users/corybaird/Desktop/graspp_2025_spring/data/examples/module_1/
['world_bank_data.csv']

Look mom this function imports data to pandas
   Unnamed: 0                     indicator country countryiso3code  date  \
0           0  GDP per capita (current US$)  Canada             CAN  2023   
1           1  GDP per capita (current US$)  Canada             CAN  2022   
2           2  GDP per capita (current US$)  Canada             CAN  2021   
3           3  GDP per capita (current US$)   Japan             JPN  2023   
4           4  GDP per capita (current US$)   Japan   

# 4. Running from command line/Powershell

In [39]:
import os
os.listdir('../../../src/examples')

['function_import_data.py',
 'oop_basic.py',
 'oop_manipulate_save.py',
 'oop_import_data_notebook.py',
 'oop_import_data.py',
 '__pycache__',
 'function_manipulate_save.py',
 'function_basic.py']

## Using terminal
<img src="screenshots/terminal.png" />


## Navigating terminal
- ``cd``: changes folders
- ``pwd``: shows current folder 
- ``ls``: Shows files
    
<img src="screenshots/pwd_ls.png" />


## Running python from terminal
- ``cd``: changes folders
- ``pwd``: shows current folder 
- ``ls``: Shows files
    
<img src="screenshots/python_run.png" />
